In [2]:
import pandas as pd
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import pickle

warnings.filterwarnings('ignore')

np.random.seed(123)

data = pd.read_csv('/Users/joelfoo/Documents/carouhack_car_price/data/sgcm-ads.csv')
data.columns

Index(['make model', 'rid', 'source_url', 'id', 'dl', 'make', 'model',
       'vehicle_type', 'car_registration_date', 'listing_price',
       'depreciation', 'category_1', 'engine_cap', 'transmission', 'mileage',
       'no_of_owners', 'coe', 'omv', 'arf', 'availability', 'posted_on',
       'last_updated_on', 'company_name', 'road_tax', 'category_2',
       'category_3'],
      dtype='object')

In [3]:
df = data[['make model', 'car_registration_date', 'coe', 'no_of_owners', 'mileage', 'arf', 'depreciation']]

In [4]:
# feature engineering

df['car_registration_date'] = pd.to_datetime(df['car_registration_date']) 
df['days_since_reg'] = (pd.to_datetime('today') - df['car_registration_date']).dt.days

df['mileage_per_year'] = df['mileage']/(df['days_since_reg']/365)
df['good_mileage'] = df['mileage_per_year']/1000 <= 15

In [56]:
# limit dataset to top 10 cars
top_10 = ['Honda Vezel 1.5A X',
          'Toyota Corolla Altis 1.6A',
          'Volkswagen Golf 1.4A TSI',
          'Toyota Wish 1.8A', 
          'BMW 5 Series 520i',
          'Mazda 5 2.0A Sunroof',
          'Volkswagen Jetta 1.4A TSI',
          'Volkswagen Scirocco 1.4A TSI',
          'Audi A4 1.8A TFSI MU',
          'Mercedes-Benz C-Class C180 Avantgarde'
         ]


df = df[(df['make model'].isin(top_10))].dropna()

model_dict = {}
for m in top_10:
    model_dict[m] = {"lower": None,
                     "upper": None,
                    "predicted": None}

55094.5 59177.0


In [68]:
state = np.random.seed(123)

for make_model in top_10:
    df_m = df[df['make model'] == make_model]
    
    mileage_processed = []
    for i in range(len(df_m['mileage'])):
        mileage = df_m['mileage'].iloc[i]
        if  mileage == 0:
            mileage_processed.append(df_m['days_since_reg'].iloc[i] / 365 * 15000)
        else:
            mileage_processed.append(mileage)

    df_m['mileage'] = mileage_processed
    
    train, test = train_test_split(df_m, test_size=0.3)

    coeff_list = ['coe', 'no_of_owners', 'arf', 'mileage', 'good_mileage', 'days_since_reg']

    x_train = train[coeff_list]
    y_train = train[['depreciation']]

    x_test = test[coeff_list]
    y_test= test[['depreciation']]
    
    #print(make_model)
    #print(y_train)
    
    # GradientBoost
    r0 = GradientBoostingRegressor(loss='quantile', alpha=0.95,
                                   n_estimators= 10,
                                   max_features = 'auto',
                                   random_state=state)
    r0.fit(x_train, y_train)
    #model_dict[make_model]["upper"] = pickle.dumps(r0)
    y_upper = r0.predict(x_test)

    r0.set_params(alpha=1-0.95)
    r0.fit(x_train,y_train)
    #model_dict[make_model]["lower"] = pickle.dumps(r0)
    y_lower = r0.predict(x_test)

    r0.set_params(loss='ls')
    r0.fit(x_train, y_train)
    #model_dict[make_model]["predicted"] = pickle.dumps(r0)
    y_pred0 = r0.predict(x_test)


    y_test['lower'] = y_lower
    y_test['pred'] = y_pred0
    y_test['upper'] = y_upper


In [7]:
f = open('/Users/joelfoo/Documents/carouhack_car_price/data/model.pkl', 'wb')
pickle.dump(model_dict, f)
f.close()

In [8]:
model_dict = pd.read_pickle('/Users/joelfoo/Documents/carouhack_car_price/data/model.pkl')

In [10]:
x_train.iloc[0:1]

,coe,no_of_owners,arf,mileage,good_mileage,days_since_reg
3254,49802,1,44135,23280,False,476


In [45]:
x_new = pd.DataFrame(np.array([123,
                               5, 2, 3, True,
                                 5]).reshape(1,6), index=np.array(range(1, 2)), columns = coeff_list)

In [49]:
x_new = pd.DataFrame(np.array([123,
                                 2, 231, 15125, True,
                                 1123]).reshape(1, 6),
                                 index=np.array(range(1, 2)),
                                 columns=coeff_list)

In [50]:
x_new

,coe,no_of_owners,arf,mileage,good_mileage,days_since_reg
1,123,2,231,15125,1,1123


In [54]:
print(pickle.loads(model_dict["Toyota Wish 1.8A"]["lower"]).predict(x_new))
print(pickle.loads(model_dict["Toyota Wish 1.8A"]["predicted"]).predict(x_new))
print(pickle.loads(model_dict["Toyota Wish 1.8A"]["upper"]).predict(x_new))

[8483.52963603]
[8559.35888768]
[11262.21448834]


In [77]:
from datetime import datetime

def calculate_months_since_reg(reg_date):
    print("calculating months since:", reg_date)
    date_format = "%d/%m/%Y"
    formatted_reg_date = datetime.strptime(reg_date, date_format)
    delta = datetime.today() - formatted_reg_date
    return delta.month



In [80]:
print()

2011-04-18 00:00:00


In [88]:
np.max(df_m['depreciation'])

18320.0

In [91]:
d = datetime.strptime('19/8/2016', "%d/%m/%Y")
d.replace(year=d.year+10)

datetime.datetime(2026, 8, 19, 0, 0)